<a href="https://colab.research.google.com/github/Letch49/ML_vvsu_2024/blob/master/lab3/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание по практике 3

##Обработка пропущенных значений

Обработайте пропущенные значения:


* Удалите пропущенные значения
* Заполните при помощи медианы, среднего для числовых значений
* Заполните при помощи моды для категориальных значений
* Интерполируйте
Выведите статистку после каждого метода и объясните


In [1]:
#!pip install scikit-learn

In [2]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from scipy.stats import spearmanr

import pandas as pd
import numpy as np

df = pd.read_csv('heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [3]:
df_dropna = df
df_dropna = df_dropna.dropna()
df_dropna = df_dropna.reset_index(drop=True)
df_dropna

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


**Примичание**

Так как нет значений NaN и количество строк не изменилось в датафрейме, то нужно добавить эти значения и уже работать с ними

In [4]:
def create_holes(dataframe, column_name, hole_percentage = 50):

    if hole_percentage < 0 or hole_percentage > 100:
        raise ValueError("Процент дырявости должен быть в диапазоне от 0 до 100.")

    dataframe_with_holes = dataframe.copy()
    num_holes = int(len(dataframe) * (hole_percentage / 100))
    hole_indices = np.random.choice(len(dataframe), num_holes, replace=False)
    dataframe_with_holes.loc[hole_indices, column_name] = np.nan

    return dataframe_with_holes

df_nan = df
df_nan = create_holes(df_nan, 'RestingBP')
df_nan

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140.0,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,NaN,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130.0,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138.0,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,NaN,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110.0,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,NaN,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130.0,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,NaN,236,0,LVH,174,N,0.0,Flat,1


**Примичание**

После добавления, видно, что добавление значений пустыми значениями прошло успешно. Можно приступать к обработке датафрейма

In [5]:
#Удаление
df_dropna = df_nan.dropna()
df_dropna = df_dropna.reset_index(drop=True)
df_dropna

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140.0,289,0,Normal,172,N,0.0,Up,0
1,37,M,ATA,130.0,283,0,ST,98,N,0.0,Up,0
2,48,F,ASY,138.0,214,0,Normal,108,Y,1.5,Flat,1
3,39,M,NAP,120.0,339,0,Normal,170,N,0.0,Up,0
4,45,F,ATA,130.0,237,0,Normal,170,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
454,44,M,ASY,120.0,169,0,Normal,144,Y,2.8,Down,1
455,41,M,ATA,120.0,157,0,Normal,182,N,0.0,Up,0
456,59,M,ASY,164.0,176,1,LVH,90,N,1.0,Flat,1
457,45,M,TA,110.0,264,0,Normal,132,N,1.2,Flat,1


In [6]:
##
#   Так как есть категориальные переменные, то нужно преобразовать их в числовые
##

df_nan_col = df.select_dtypes(['object']).columns
df_nan[df_nan_col] = df_nan[df_nan_col].apply( lambda x: pd.factorize (x)[0])


##
#   Ограничение знаков после запятой
##

pd.options.display.float_format = '{:.5}'.format

##
#   После этого уже можно работать с датавреймом
##

In [7]:
#Заполнение при помощи медианы

df_median = df_nan.fillna(df_nan.median())
df_median

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140.0,289,0,0,172,0,0.0,0,0
1,49,1,1,130.0,180,0,0,156,0,1.0,1,1
2,37,0,0,130.0,283,0,1,98,0,0.0,0,0
3,48,1,2,138.0,214,0,0,108,1,1.5,1,1
4,54,0,1,130.0,195,0,0,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,0,3,110.0,264,0,0,132,0,1.2,1,1
914,68,0,2,130.0,193,1,0,141,0,3.4,1,1
915,57,0,2,130.0,131,0,0,115,1,1.2,1,1
916,57,1,0,130.0,236,0,2,174,0,0.0,1,1


In [8]:
#Заполнение при помощи среднего

df_mean = df_nan.fillna(df_nan.mean())
df_mean

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140.0,289,0,0,172,0,0.0,0,0
1,49,1,1,132.71,180,0,0,156,0,1.0,1,1
2,37,0,0,130.0,283,0,1,98,0,0.0,0,0
3,48,1,2,138.0,214,0,0,108,1,1.5,1,1
4,54,0,1,132.71,195,0,0,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,0,3,110.0,264,0,0,132,0,1.2,1,1
914,68,0,2,132.71,193,1,0,141,0,3.4,1,1
915,57,0,2,130.0,131,0,0,115,1,1.2,1,1
916,57,1,0,132.71,236,0,2,174,0,0.0,1,1


In [9]:
#Заполнение при помощи моды

df_mode = df_nan.fillna(df_nan['RestingBP'].mode()[0])
df_mode

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140.0,289,0,0,172,0,0.0,0,0
1,49,1,1,120.0,180,0,0,156,0,1.0,1,1
2,37,0,0,130.0,283,0,1,98,0,0.0,0,0
3,48,1,2,138.0,214,0,0,108,1,1.5,1,1
4,54,0,1,120.0,195,0,0,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,0,3,110.0,264,0,0,132,0,1.2,1,1
914,68,0,2,120.0,193,1,0,141,0,3.4,1,1
915,57,0,2,130.0,131,0,0,115,1,1.2,1,1
916,57,1,0,120.0,236,0,2,174,0,0.0,1,1


In [10]:
#Заполнение при помощи интерполяции

df_interpolate = df_nan.fillna(df_nan.interpolate())
df_interpolate

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140.0,289,0,0,172,0,0.0,0,0
1,49,1,1,135.0,180,0,0,156,0,1.0,1,1
2,37,0,0,130.0,283,0,1,98,0,0.0,0,0
3,48,1,2,138.0,214,0,0,108,1,1.5,1,1
4,54,0,1,129.0,195,0,0,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,0,3,110.0,264,0,0,132,0,1.2,1,1
914,68,0,2,120.0,193,1,0,141,0,3.4,1,1
915,57,0,2,130.0,131,0,0,115,1,1.2,1,1
916,57,1,0,130.0,236,0,2,174,0,0.0,1,1


**Объясните**

**Ответ**

Основная проблема заполнения пустых значений - это искажение статистики. Данные способы применимы при небольшом количестве пропусков. Одна большой процент пустых значений, может повлечь за собой искажение данных и если они в разных местах, то удаление так же может повлечь на конечный результат. Нужно понимать для чего это нужно. Методы заполнения медианой и средним значениям приблизительны и способны боле-менее помочб в будущем с обработкой данных.

**Те, кто выполняет задания со * делают поиск выбросов с помощью написанного ранее класса**

Найти выбросы по числовой переменной (использовать метод основанный на среднем и на межквартильном размахе). Объясните результат

Вывести статистику с выбросами и без. Сравните и объясните результат


In [11]:
class Outline:
  def __init__(self, df):
    self.df = df

  # На основе среднего
  def filter_outliers_by_mean_std(self, c):

    outlier_id = []

    mean = df[c].mean()
    std =  df[c].std()
    current_col_series = df[c].items()

    cut_min = mean-std*3
    cut_max = mean+std*3
    
    # index, value
    for index, value in current_col_series:
      if value <= cut_min or value >= cut_max:
        outlier_id.append(index)
        
    return outlier_id

  # Межквартильный размах
  def filter_outliers_by_median(self, c):

    df_clear = []
    outlier_id = []

    q1 = df[c].quantile(.25)
    q3 = df[c].quantile(.75)

    out_min = q1-1.5*(q3-q1)
    out_max = q3+1.5*(q3-q1)

    current_col_series = df[c].items()

    # index, value
    for index, value in current_col_series:
      if value <= out_min or value >= out_max:
        outlier_id.append(index)

    return outlier_id

In [12]:
#Методом среднего
df_outliers = Outline(df)
df_out = df
df_outliers_drop = df_outliers.filter_outliers_by_mean_std('RestingBP')
df_outliers_clear = df_out.drop(index=df_outliers_drop)
df_outliers_clear = df_outliers_clear.reset_index(drop=True)
df_outliers_clear

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
905,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
906,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
907,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
908,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [13]:
#При помощи межквартильного размаха
df_out2 = df
df_outliers_drop = df_outliers.filter_outliers_by_median('RestingBP')
df_outliers_clear2 = df_out2.drop(index=df_outliers_drop)
df_outliers_clear2 = df_outliers_clear2.reset_index(drop=True)
df_outliers_clear2

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
871,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
872,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
873,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
874,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [14]:
#Статистика и объяснение
print(f'1. Оригинальный\n{df.describe()}\n\n2. Метод среднего\n{df_outliers_clear.describe()}\n\n3. Межквартильный размах\n{df_outliers_clear2.describe()}')


1. Оригинальный
         Age  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak  HeartDisease
count  918.0      918.0        918.0      918.0  918.0    918.0         918.0
mean  53.511      132.4        198.8    0.23312 136.81  0.88736       0.55338
std   9.4326     18.514       109.38    0.42305  25.46   1.0666       0.49741
min     28.0        0.0          0.0        0.0   60.0     -2.6           0.0
25%     47.0      120.0       173.25        0.0  120.0      0.0           0.0
50%     54.0      130.0        223.0        0.0  138.0      0.6           1.0
75%     60.0      140.0        267.0        0.0  156.0      1.5           1.0
max     77.0      200.0        603.0        1.0  202.0      6.2           1.0

2. Метод среднего
         Age  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak  HeartDisease
count  910.0      910.0        910.0      910.0  910.0    910.0         910.0
mean  53.493     132.05       199.12    0.23187 136.81  0.88363       0.55055
std   9.4483     17.178      

**Объяснения**

**Ответ**

Сравнивая исходный датафрейм с методом среднего значения, видно, что было найдено 8 выбросов на основе среднего и отклонения. В случае медквартильного рахмаха, то видно, что метод нашел элементы, которые далеки от интервала квартиля [Q25, Q75], являются выбросом, от чего количество элементов больше, чем при выполнении метода среднего.

##Дубликаты

Проверьте датафрейм на дубликаты и удалите их, если дублей нет - создайте.

In [15]:
#Проверка на дубли
df_duplicated = df[df.duplicated()]
df_duplicated

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [16]:
##
#   Создание дупликатов
##

df = pd.concat((df, df.loc[0:100]))
df_duplicated = df[df.duplicated()]
df_duplicated

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
96,43,M,ATA,142,207,0,Normal,138,N,0.0,Up,0
97,39,M,NAP,160,147,1,Normal,160,N,0.0,Up,0
98,56,M,ASY,120,85,0,Normal,140,N,0.0,Up,0
99,41,M,ATA,125,269,0,Normal,144,N,0.0,Up,0


In [17]:
#Удаление дублей

df = df.drop_duplicates()
df_no_duplicates = df[df.duplicated()]
df_no_duplicates

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


## Преобразование признаков

Масштабируйте числовые признаки через MinMaxScaler, StandardScaler

In [18]:
num_features = df.select_dtypes(include=['float64']).columns
df_oldpeak = df.Oldpeak.unique()

print(f'{num_features}\n\n{df_oldpeak}')

Index(['Oldpeak'], dtype='object')

[ 0.   1.   1.5  2.   3.   4.   0.5  2.5  5.   0.8  0.7  1.4  2.1  0.4
  0.2  1.7  2.2  0.1  1.6  1.3  0.3  1.8  2.6 -0.9  2.8 -2.6 -1.5 -0.1
  0.9  1.1  2.4 -1.  -1.1 -0.7 -0.8  3.7  1.2 -0.5 -2.   1.9  3.5  0.6
  3.1  2.3  3.4  3.6  4.2  3.2  5.6  3.8  2.9  6.2  4.4]


In [19]:
# MinMaxScaler

scaler = MinMaxScaler()
df_scaler = df.copy()
df_scaler[num_features] = scaler.fit_transform(df[num_features])
df_scaler.Oldpeak.unique()

array([0.29545455, 0.40909091, 0.46590909, 0.52272727, 0.63636364,
       0.75      , 0.35227273, 0.57954545, 0.86363636, 0.38636364,
       0.375     , 0.45454545, 0.53409091, 0.34090909, 0.31818182,
       0.48863636, 0.54545455, 0.30681818, 0.47727273, 0.44318182,
       0.32954545, 0.5       , 0.59090909, 0.19318182, 0.61363636,
       0.        , 0.125     , 0.28409091, 0.39772727, 0.42045455,
       0.56818182, 0.18181818, 0.17045455, 0.21590909, 0.20454545,
       0.71590909, 0.43181818, 0.23863636, 0.06818182, 0.51136364,
       0.69318182, 0.36363636, 0.64772727, 0.55681818, 0.68181818,
       0.70454545, 0.77272727, 0.65909091, 0.93181818, 0.72727273,
       0.625     , 1.        , 0.79545455])

In [20]:
# StardartScaler

scaler_std = StandardScaler()
df_scaler_std = df.copy()
df_scaler_std[num_features] = scaler_std.fit_transform(df[num_features])
df_scaler_std.Oldpeak.unique()

array([-0.83243239,  0.10566353,  0.57471149,  1.04375945,  1.98185537,
        2.91995129, -0.36338443,  1.51280741,  3.85804721, -0.08195566,
       -0.17576525,  0.4809019 ,  1.13756904, -0.45719402, -0.64481321,
        0.76233067,  1.23137863, -0.7386228 ,  0.66852108,  0.3870923 ,
       -0.55100362,  0.85614026,  1.606617  , -1.67671872,  1.79423618,
       -3.27148179, -2.23957627, -0.92624199,  0.01185394,  0.19947312,
        1.41899782, -1.77052831, -1.86433791, -1.48909954, -1.58290913,
        2.63852251,  0.29328271, -1.30148035, -2.70862423,  0.94994986,
        2.45090333, -0.26957484,  2.07566496,  1.32518822,  2.35709374,
        2.54471292,  3.10757047,  2.16947455,  4.42090476,  2.73233211,
        1.88804578,  4.98376231,  3.29518966])

In [21]:
df_scaled = df.copy()
df_scaled[num_features] = df_scaler_std[num_features]

df_scaled

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,-0.83243,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,0.10566,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,-0.83243,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,0.57471,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,-0.83243,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,0.29328,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,2.3571,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,0.29328,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,-0.83243,Flat,1


Масштабируйте категориальные признаки через OneHotEncoder

In [22]:
# Объявление категориальных признаков
df = df.astype({'Sex': 'category',
                'ChestPainType': 'category',
                'RestingECG': 'category',
                'ExerciseAngina': 'category',
                'ST_Slope': 'category',
                })
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Age             918 non-null    int64   
 1   Sex             918 non-null    category
 2   ChestPainType   918 non-null    category
 3   RestingBP       918 non-null    int64   
 4   Cholesterol     918 non-null    int64   
 5   FastingBS       918 non-null    int64   
 6   RestingECG      918 non-null    category
 7   MaxHR           918 non-null    int64   
 8   ExerciseAngina  918 non-null    category
 9   Oldpeak         918 non-null    float64 
 10  ST_Slope        918 non-null    category
 11  HeartDisease    918 non-null    int64   
dtypes: category(5), float64(1), int64(6)
memory usage: 62.6 KB


In [23]:
cat_features = df.select_dtypes(include=['category']).columns

df_cat_features = pd.get_dummies(df, columns=cat_features, drop_first=True)

cat_features

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')

In [24]:
encoder = OneHotEncoder()

encode_features = encoder.fit_transform(df[cat_features])

encode_col = encoder.get_feature_names_out(cat_features)
encode_col

array(['Sex_F', 'Sex_M', 'ChestPainType_ASY', 'ChestPainType_ATA',
       'ChestPainType_NAP', 'ChestPainType_TA', 'RestingECG_LVH',
       'RestingECG_Normal', 'RestingECG_ST', 'ExerciseAngina_N',
       'ExerciseAngina_Y', 'ST_Slope_Down', 'ST_Slope_Flat',
       'ST_Slope_Up'], dtype=object)

##Задание для гениев*


Реализовать расчет корреляции Спирмана

In [25]:
#Функция по расчету тут

##
#  Реализаци спиртама через библиотеку SciPy
##

def scipy_spearman(col1, col2):

    p, rho = spearmanr(df[col1], df[col2])

    return p


##
#  Реализация спирмана через Pandas, используя метод pd.corr(method='spearman')
##

def pandas_spearman(col1, col2):
    corr_spearman = df.select_dtypes(np.number)
    corr_spearman = corr_spearman[[col1, col2]].corr(method='spearman')

    return corr_spearman


# Используя метод Спирмана, берут две выборки. Это на случай использования матрицы корреляции по методу Спирмена

#def pandas_spearman(df_corr, col1 = None, col2 = None):
#
#    if col1 is None and col2 is None:
#        corr_spearman = df_corr.select_dtypes(np.number)
#        corr_spearman = corr_spearman.corr(method='spearman')
#
#        return corr_spearman
#
#    else:
#        corr_spearman = df_corr[[col1, col2]].corr(method='spearman')
#        return corr_spearman

In [26]:
#Пример использования тут и объяснение принципа работы
scipy_spearman('Age', 'RestingBP')

0.279390076451401

In [27]:
pandas_spearman('Age', 'RestingBP')

,Age,RestingBP
Age,1.0,0.27939
RestingBP,0.27939,1.0


**Ответ**

Коэфицент ранговой корреляции Спирмана используется для измерения между величинами не по численному значению, а по ранговой системе. Используется система ранжирования для каждой выборке, либо по возрастанию, либо по убыванию, после чего считается по формуле:

![Alt text](image-1.png) - вычисление квадрата разности между рангами 

![Alt text](image.png) - вычисление коэффицциента  ранговой корреляции, где *n* - объем выборки (количество рангов)

Однако, если при вычислении выборки, ранговое значение будет дробное из-за одинакового числового значения, то коэффициент будет высчитываться приблизительно и это необходимо учитывать.

По характеристике тесноты связи между двумя выборками определяется его сила связи (см. таблицу ниже)

| Количественная мера тесноты связи|Характеристика силы связи |
|---------------------|---------------------------|
|0.0|Нулевая|
|0.1/-0.1|Слабая/Отрицательно слабая|
|0.3/-0.3|Умеренная/Отрицательно умеренная|
|0.5/-0.5|Средняя/Отрицательно средняя|
|0.9/-0.9|Высокая/Отрицательно высокая|
|1.0/-1.0|Очень высокая/Отрицательно очень высокая|